In [53]:
import urllib
import pymysql
import os
import pandas as pd

from google.cloud import storage
from urllib.parse import urlparse

In [109]:
project_id = 'optical-scarab-285012'
bucket_name = 'mle-project'
path = 'Images/Thumbnails/'
storage_client = storage.client.Client(project=project_id)

In [111]:
def upload_blob(bucket_name, url, path, prefix):   
    file = urllib.request.urlopen(url)
    content_type = file.info().get_content_type()
    file_name = os.path.basename(urlparse(url).path)
    dest = path + str(prefix) + '-' + file_name
    
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(dest)

    blob.upload_from_string(file.read(), content_type=content_type)
    
    return dest

In [106]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='localhost',
    port=int(3306),
    user="root",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()

def query_url():
    '''
    
    '''
    query = '''
    SELECT id, image
    FROM project.ProductHeader
    WHERE image like 'http%';;
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY INSERT:', err)
    result = cur.fetchall()
    return result

def insert_filepath(id, url, bucket, bucket_path):
    '''
    
    '''
    query = f'''
    INSERT INTO Files(id, url, bucket, bucket_path)
    VALUES('{id}', '{url}', '{bucket}', '{bucket_path}');
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY INSERT:', err)

In [89]:
df = pd.DataFrame(query_url(), columns=['id', 'url'])

In [90]:
df = df[:5]

In [110]:
for i in df.values:
    id = i[0]
    url = i[1]
    filepath = upload_blob(bucket_name, url, path, id)
    print('Uploaded' + str(id))
    insert_filepath(id, url, bucket_name, filepath)
    print('Recorded' + str(id))

Uploaded1892
Recorded1892
Uploaded1893
Recorded1893
Uploaded1894
Recorded1894
Uploaded1895
Recorded1895
Uploaded1896
Recorded1896
